In [71]:
import cantera as ct
from omnisoot import ConstantVolumeReactor, SootGas, SootThermo, constants
import numpy as np
import os
import pandas as pd
import time
import matplotlib.pyplot as plt

In [72]:
### gas
gas = ct.Solution("data/Caltech_basic.yaml");
soot_gas = SootGas(gas);
### ---------------------------------------------------------------------------------
### Temperature Time History
time_data = [0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05, 0.055, 0.06, 0.065, 0.07, 0.075, 0.08, 0.085, 0.09, 0.095, 0.1];
temp_data = [2200, 2100, 2000, 1900, 1800, 1700, 1672, 1671, 1670, 1669, 1666, 1662, 1655, 1643, 1623, 1590, 1536, 1448, 1301, 1060, 663];
fixed_profile = np.array([time_data, Temp_data])

In [73]:
### Intial conditions of the reactor
T = temp_data[0]; # K
P = 101350; # Pa
X = {'CH4':0.2, 'N2': 0.8};
max_res_time = 0.1
### Initial conditions
soot_gas.TPX = T, P, X;

In [74]:
### Initial soot properties 
N_agg_number_0 = 1e20; # 1/m3
n_p_0 = 1;
d_p_0 = 5e-9; # m
H_C_ratio = 0.8; 
### Calculating soot array to use as an input to the model
N_agg_0 = N_agg_number_0 / soot_gas.density / constants.Av; # mol/kg
N_pri_0 = N_agg_0 * n_p_0; # mol/kg
m_p_0 = constants.Pi / 6 * d_p_0 ** 3.0 * constants.rho_soot
C_tot_0 = N_pri_0 * m_p_0;
H_tot_0 = H_C_ratio * C_tot_0;

In [75]:
### Reactor
reactor = ConstantVolumeReactor(soot_gas);
soot = reactor.soot;
### PAH growth models
PAH_growth_model_type = ["ReactiveDimerization", "DimerCoalescence", "EBridgeModified", "IrreversibleDimerization"][0];
soot.PAH_growth_model_type = PAH_growth_model_type;
### Setting precursors
soot.set_precursor_names(['A2', 'A2R5', 'A3R5', 'A4R5']);
### Reactor integrator settings
reactor.max_step = 1e-3;
reactor.max_time = max_res_time;
reactor.solver_type = "BDF";
reactor.temperature_solver_type = "profile_time";
reactor.set_fix_temperature_profile(fixed_profile);
### Initial soot
reactor.initial_soot_type = "custom";
reactor.user_defined_initial_soot = np.array([N_agg_0, N_pri_0, C_tot_0, H_tot_0]);
### Flags
soot.inception_enabled = False;
soot.PAH_adsorption_enabled = False;
soot.surface_growth_enabled = False;
soot.oxidation_enabled = False;
soot.coagulation_enabled = True;
reactor.start();

In [76]:
### Soltuion array
states = ct.SolutionArray(gas, 1, extra = {  'restime': [0.0] ,
                                             'N_agg': [soot.N_agg], 'N_pri': [soot.N_pri], 
                                             'C_tot': [soot.C_tot], 'H_tot': [soot.H_tot],
                                             'A_tot': [soot.A_tot],
                                             'd_p': [soot.d_p], 'd_m': [soot.d_m],
                                             'd_v': [soot.d_v], 'd_g': [soot.d_g],
                                             'n_p': [soot.n_p],
                                             'total_mass' : [soot.total_mass],
                                             'volume_fraction':[soot.volume_fraction],
                                             'SSA': [soot.SSA],
                                             'inception_mass': [soot.inception_mass],
                                             'coagulation_mass': [soot.coagulation_mass],
                                             'PAH_adsorption_mass' : [soot.PAH_adsorption_mass],
                                             'surface_growth_mass' : [soot.surface_growth_mass],
                                             'oxidation_mass': [soot.oxidation_mass],
                                             'total_carbon_mass' : [reactor.total_carbon_mass],
                                             'total_hydrogen_mass' : [reactor.total_hydrogen_mass],
                                             'soot_carbon_mass' : [reactor.soot_carbon_mass],
                                        });
def append():
    states.append(gas.state, restime = reactor.restime, 
              N_agg = soot.N_agg, N_pri =soot.N_pri,
              C_tot = soot.C_tot, H_tot = soot.H_tot, A_tot = soot.A_tot,
              d_p = soot.d_p, d_m = soot.d_m, d_v = soot.d_v, d_g = soot.d_g,
              n_p = soot.n_p,
              total_mass = soot.total_mass, volume_fraction = soot.volume_fraction, SSA = soot.SSA,
              inception_mass = soot.inception_mass,
              coagulation_mass = soot.coagulation_mass,
              PAH_adsorption_mass = soot.PAH_adsorption_mass,
              surface_growth_mass = soot.surface_growth_mass,
              oxidation_mass = soot.oxidation_mass,
              total_carbon_mass = reactor.total_carbon_mass,
              total_hydrogen_mass = reactor.total_hydrogen_mass,
              soot_carbon_mass = reactor.soot_carbon_mass                
             );

In [77]:
### Running the reactor
start_time = time.time()
step = 0;
while reactor.restime < max_res_time:
    step += 1;
    reactor.step();
    if step % 5 == 0:
        append();
else:
    append();
end_time = time.time();
print(f"the simulation took {(end_time-start_time):0.4f} s");

the simulation took 2.1899 s
